# Task 1: User-based Collaborative Filtering


In [1]:
#import libraries.

import pandas as pd
import numpy as np

In [2]:
cd ml-1m 

/Users/issaczerihun/Desktop/untitled folder/ml-1m


In [3]:
ratingColumnNames = ['User ID', 'Movie ID', 'Rating', 'Timestamp']

In [4]:
filename = 'ratings.csv'

In [5]:
#Step1: reading ratings.csv file
ratings_df = pd.read_csv(filename, sep='::', names = ratingColumnNames, engine='python')

In [6]:
ratings_df.head()

,User ID,Movie ID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [7]:
ratings_df.shape

(1000209, 4)

In [8]:
# Step 2: Obtaining the number of total users and items.
#Number of movies ranging from 1 - 3952
max_movie_id = ratings_df['Movie ID'].max()
print("Max movie ID in dataset:", max_movie_id)

Max movie ID in dataset: 3952


In [9]:

# You'll need this in order build user-item matrices.
n_users = ratings_df["User ID"].nunique()
n_movies = max_movie_id


In [10]:
print(f"Num users: {n_users}")
print(f"Num movies: {n_movies}")

Num users: 6040
Num movies: 3952


In [11]:
# Generate a random number between 0 and 6039 (number of users)
import random
random.seed(3)
random_user = random.randint(0, 6039)


In [12]:
print("Random user: User", random_user)

Random user: User 1949


In [13]:
# Step 3: Next we want to split the dataset into training and testing parts.
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(ratings_df, random_state = 3)

In [14]:
train_df

,User ID,Movie ID,Rating,Timestamp
70637,476,2262,2,976222807
641902,3859,338,3,965927649
468924,2888,667,2,972061868
12361,93,2916,4,977631699
993177,6000,2135,2,956884496
...,...,...,...,...
572344,3507,2710,2,966999148
452227,2785,2458,2,972990850
601337,3654,1079,4,966458221
722584,4327,2628,3,965244573


In [15]:
test_df

,User ID,Movie ID,Rating,Timestamp
7961,53,2872,5,977981423
270019,1635,1394,4,974771697
773462,4610,1544,2,984859886
600962,3653,3793,5,966453201
384987,2249,2706,5,974596882
...,...,...,...,...
387043,2263,3177,3,974582457
675656,4053,2001,4,965491688
863563,5200,1678,4,961648057
892619,5393,1956,5,960320999


In [16]:
# Step 4: Build user items matrices for both training and testing datasets. 

trainingDataset = np.zeros((n_users,n_movies))
testingDataset = np.zeros((n_users,n_movies))

In [17]:
#Populate the matrix with ratings from training dataset

# First row (check trainingDataset) with user id 476 and movie id 2262.
# Rated value (3) is going to be marked at matrix value (476-1,2262-1) (* don't forget, starts at 0).
# Continues on for whole of training dataset. 

for row in train_df.itertuples():
    trainingDataset[row[1]-1, row[2]-1] = row[3]
trainingDataset = pd.DataFrame(trainingDataset)

In [18]:
#Populate the matrix with ratings from testing dataset
for row in test_df.itertuples():
    testingDataset[row[1]-1, row[2]-1] = row[3]
testingDataset = pd.DataFrame(testingDataset)

In [19]:
trainingDataset, testingDataset

(      0     1     2     3     4     5     6     7     8     9     ...  3942  \
 0      5.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
 1      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
 2      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
 3      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
 4      0.0   0.0   0.0   0.0   0.0   2.0   0.0   0.0   0.0   0.0  ...   0.0   
 ...    ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   ...   
 6035   0.0   0.0   0.0   0.0   0.0   3.0   0.0   0.0   0.0   0.0  ...   0.0   
 6036   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
 6037   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
 6038   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
 6039   3.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
 
       3943  3944  3945  3946  3947  3

In [20]:
#Generating a random movie
random.seed(9)
random_movie = random.randint(0, n_movies - 1)

In [21]:
print("Random movie: ID", random_movie)

Random movie: ID 1896


In [22]:
# Checking to see if anyone has rated the movie.
trainingDataset[1896].sum()

344.0

In [23]:
import numpy as np

# Create an empty 2D array to store Jaccard similarities
jaccard_similarity_score = np.zeros((len(trainingDataset), len(trainingDataset)))
# Nested loop that will store the similarity scores of every pair of users.
for i, user_i in enumerate(trainingDataset.values):
    for j, user_j in enumerate(trainingDataset.values):

        # Create sets of movies where the user i and j rated a movie above a zero. 
        set_i = set(np.where(user_i > 0)[0])
        set_j = set(np.where(user_j > 0)[0])

        # Calculate the intersection and union of the sets
        intersection = len(set_i.intersection(set_j))
        union = len(set_i.union(set_j))

        # Calculating Jaccard similarity
        if union == 0:
             jaccard_score = 0.0
            # This is done to avoid zero division
        else:
            jaccard_score = intersection / union

        # Store the Jaccard similarity in the array
        jaccard_similarity_score[i][j] = jaccard_score

# jaccard_similarity_score now contains the Jaccard similarities between all pairs of users

In [24]:
jaccard_similarity_score

array([[1.        , 0.01492537, 0.04109589, ..., 0.        , 0.10084034,
        0.02310231],
       [0.01492537, 1.        , 0.04065041, ..., 0.02857143, 0.03977273,
        0.07100592],
       [0.04109589, 0.04065041, 1.        , ..., 0.0212766 , 0.06034483,
        0.03780069],
       ...,
       [0.        , 0.02857143, 0.0212766 , ..., 1.        , 0.01980198,
        0.02545455],
       [0.10084034, 0.03977273, 0.06034483, ..., 0.01980198, 1.        ,
        0.08181818],
       [0.02310231, 0.07100592, 0.03780069, ..., 0.02545455, 0.08181818,
        1.        ]])

In [25]:
# Following code inspired from item_KNN_based_CF.ipynb -> courtesy of RMIT University.
K = 1
EPSILON = 1e-9
# Create an empty array to store predictions for user 1949
target_user_predictions = np.zeros_like(testingDataset.values[random_user])

for j in range(n_movies):
    if testingDataset.values[random_user, j] > 0:
        # Find top-K most similar users to the target user, excluding itself
        similar_user_ids = np.argsort(jaccard_similarity_score[random_user])[-(K + 1):-1]

        # Generates the coefficient values of similar users
        similar_values = jaccard_similarity_score[random_user][similar_user_ids]

        # Calculates the average value of the target user's ratings
        similar_users = trainingDataset.values[similar_user_ids]
        user_mean = np.sum(trainingDataset.values[random_user]) / (np.sum(np.clip(trainingDataset.values[random_user], 0, 1)) + EPSILON)
        similar_user_mean = np.sum(similar_users, axis=1) / (np.sum(np.clip(similar_users, 0, 1), axis=1) + EPSILON)

        # Select the users who rated item j
        users_who_rated_J = similar_users[:, j] > 0

        sum_mean = similar_values[users_who_rated_J] * (similar_users[users_who_rated_J, j] - similar_user_mean[users_who_rated_J])

        # Filter unrated items
        weight = np.clip(similar_users[users_who_rated_J, j], 0, 1)
        sum_mean *= weight

        # Calculate the predicted rating for item j by the target user
        target_user_predictions[j] = user_mean + np.sum(sum_mean) / (np.sum(similar_values[users_who_rated_J]) + EPSILON)
        target_user_predictions[j] = np.clip(target_user_predictions[j], 0, 5)

# Actual ratings for the target user
actual_ratings = testingDataset.values[random_user, :]

from sklearn.metrics import mean_squared_error
# Calculate RMSE for the target user's ratings
mse = mean_squared_error(actual_ratings, target_user_predictions)
rmse = np.sqrt(mse)

print(f"RMSE for user {random_user} when K = {K}: {rmse}")


RMSE for user 1949 when K = 1: 0.13126144600468845


In [26]:
# Above code repeated for k values 10, 20, 30 and 50. 
# Following code inspired from item_KNN_based_CF.ipynb -> courtesy of RMIT.
K = 10
EPSILON = 1e-9

for j in range(n_movies):
    if testingDataset.values[random_user, j] > 0:

        similar_user_ids = np.argsort(jaccard_similarity_score[random_user])[-(K + 1):-1]

        similar_values = jaccard_similarity_score[random_user][similar_user_ids]

        similar_users = trainingDataset.values[similar_user_ids]
        user_mean = np.sum(trainingDataset.values[random_user]) / (np.sum(np.clip(trainingDataset.values[random_user], 0, 1)) + EPSILON)
        similar_user_mean = np.sum(similar_users, axis=1) / (np.sum(np.clip(similar_users, 0, 1), axis=1) + EPSILON)

        users_who_rated_J = similar_users[:, j] > 0
        
        sum_mean = similar_values[users_who_rated_J] * (similar_users[users_who_rated_J, j] - similar_user_mean[users_who_rated_J])

        weight = np.clip(similar_users[users_who_rated_J, j], 0, 1)
        sum_mean *= weight

        target_user_predictions[j] = user_mean + np.sum(sum_mean) / (np.sum(similar_values[users_who_rated_J]) + EPSILON)
        target_user_predictions[j] = np.clip(target_user_predictions[j], 0, 5)


actual_ratings = testingDataset.values[random_user, :]

mse = mean_squared_error(actual_ratings, target_user_predictions)
rmse = np.sqrt(mse)

print(f"RMSE for user {random_user} when K = {K}: {rmse}")



RMSE for user 1949 when K = 10: 0.11350182362972026


In [27]:
# Following code inspired from item_KNN_based_CF.ipynb -> courtesy of RMIT.
K = 20
EPSILON = 1e-9

target_user_predictions = np.zeros_like(testingDataset.values[random_user])

for j in range(n_movies):
    if testingDataset.values[random_user, j] > 0:

        similar_user_ids = np.argsort(jaccard_similarity_score[random_user])[-(K + 1):-1]

        similar_values = jaccard_similarity_score[random_user][similar_user_ids]

        similar_users = trainingDataset.values[similar_user_ids]
        user_mean = np.sum(trainingDataset.values[random_user]) / (np.sum(np.clip(trainingDataset.values[random_user], 0, 1)) + EPSILON)
        similar_user_mean = np.sum(similar_users, axis=1) / (np.sum(np.clip(similar_users, 0, 1), axis=1) + EPSILON)

        users_who_rated_J = similar_users[:, j] > 0
        
        sum_mean = similar_values[users_who_rated_J] * (similar_users[users_who_rated_J, j] - similar_user_mean[users_who_rated_J])

        weight = np.clip(similar_users[users_who_rated_J, j], 0, 1)
        sum_mean *= weight

        target_user_predictions[j] = user_mean + np.sum(sum_mean) / (np.sum(similar_values[users_who_rated_J]) + EPSILON)
        target_user_predictions[j] = np.clip(target_user_predictions[j], 0, 5)


actual_ratings = testingDataset.values[random_user, :]

mse = mean_squared_error(actual_ratings, target_user_predictions)
rmse = np.sqrt(mse)

print(f"RMSE for user {random_user} when K = {K}: {rmse}")

RMSE for user 1949 when K = 20: 0.12582975967352836


In [28]:
# Following code inspired from item_KNN_based_CF.ipynb -> courtesy of RMIT.
K = 30
EPSILON = 1e-9

target_user_predictions = np.zeros_like(testingDataset.values[random_user])

for j in range(n_movies):
    if testingDataset.values[random_user, j] > 0:
        similar_user_ids = np.argsort(jaccard_similarity_score[random_user])[-(K + 1):-1]

        similar_values = jaccard_similarity_score[random_user][similar_user_ids]

        similar_users = trainingDataset.values[similar_user_ids]
        user_mean = np.sum(trainingDataset.values[random_user]) / (np.sum(np.clip(trainingDataset.values[random_user], 0, 1)) + EPSILON)
        similar_user_mean = np.sum(similar_users, axis=1) / (np.sum(np.clip(similar_users, 0, 1), axis=1) + EPSILON)

        users_who_rated_J = similar_users[:, j] > 0
        
        sum_mean = similar_values[users_who_rated_J] * (similar_users[users_who_rated_J, j] - similar_user_mean[users_who_rated_J])

        weight = np.clip(similar_users[users_who_rated_J, j], 0, 1)
        sum_mean *= weight

        target_user_predictions[j] = user_mean + np.sum(sum_mean) / (np.sum(similar_values[users_who_rated_J]) + EPSILON)
        target_user_predictions[j] = np.clip(target_user_predictions[j], 0, 5)

actual_ratings = testingDataset.values[random_user, :]

mse = mean_squared_error(actual_ratings, target_user_predictions)
rmse = np.sqrt(mse)

print(f"RMSE for user {random_user} when K = {K}: {rmse}")

RMSE for user 1949 when K = 30: 0.1163237133796566


In [29]:
# Following code inspired from item_KNN_based_CF.ipynb -> courtesy of RMIT.
K = 50
EPSILON = 1e-9

target_user_predictions = np.zeros_like(testingDataset.values[random_user])

for j in range(n_movies):
    if testingDataset.values[random_user, j] > 0:
        similar_user_ids = np.argsort(jaccard_similarity_score[random_user])[-(K + 1):-1]

        similar_values = jaccard_similarity_score[random_user][similar_user_ids]

        similar_users = trainingDataset.values[similar_user_ids]
        user_mean = np.sum(trainingDataset.values[random_user]) / (np.sum(np.clip(trainingDataset.values[random_user], 0, 1)) + EPSILON)
        similar_user_mean = np.sum(similar_users, axis=1) / (np.sum(np.clip(similar_users, 0, 1), axis=1) + EPSILON)

        users_who_rated_J = similar_users[:, j] > 0
        
        sum_mean = similar_values[users_who_rated_J] * (similar_users[users_who_rated_J, j] - similar_user_mean[users_who_rated_J])

        weight = np.clip(similar_users[users_who_rated_J, j], 0, 1)
        sum_mean *= weight

        target_user_predictions[j] = user_mean + np.sum(sum_mean) / (np.sum(similar_values[users_who_rated_J]) + EPSILON)
        target_user_predictions[j] = np.clip(target_user_predictions[j], 0, 5)

actual_ratings = testingDataset.values[random_user, :]

mse = mean_squared_error(actual_ratings, target_user_predictions)
rmse = np.sqrt(mse)

print(f"RMSE for user {random_user} when K = {K}: {rmse}")

RMSE for user 1949 when K = 50: 0.11259699576782761


# Task 2: Item-based Filtering

# Jaccard similarity metric.

In [30]:
# An array is created to store Jaccard similarity metrics based on movies
np_movie_jaccard_similarity = np.zeros((n_movies, n_movies))
# Nested loop that will calc and store the similarity scores of every pair of users.
# Item based filtering so the training dataset matrix must be transposed.
for i, item_i in enumerate(trainingDataset.T.values):
    for j, item_j in enumerate(trainingDataset.T.values):
        
        #Getting the users that rated each movie
        users_rated_i = set(np.where(item_i > 0)[0])
        users_rated_j = set(np.where(item_j > 0)[0])
        #Getting the intersection and the union of these retings
        intersection = len(users_rated_i.intersection(users_rated_j))
        union = len(users_rated_i.union(users_rated_j))
        #Calclulating the similarity score.
        if union == 0:
            jaccard_similarity = 0.0
        else:
            jaccard_similarity = intersection / union
        #Storing the similarity in the array. 
        np_movie_jaccard_similarity[i][j] = jaccard_similarity

In [31]:
np_movie_jaccard_similarity

array([[1.        , 0.14918676, 0.0831899 , ..., 0.01028939, 0.00513809,
        0.05290397],
       [0.14918676, 1.        , 0.09746835, ..., 0.01851852, 0.00369004,
        0.0546875 ],
       [0.0831899 , 0.09746835, 1.        , ..., 0.0156658 , 0.0026178 ,
        0.04677419],
       ...,
       [0.01028939, 0.01851852, 0.0156658 , ..., 1.        , 0.08196721,
        0.06410256],
       [0.00513809, 0.00369004, 0.0026178 , ..., 0.08196721, 1.        ,
        0.04153355],
       [0.05290397, 0.0546875 , 0.04677419, ..., 0.06410256, 0.04153355,
        1.        ]])

In [32]:
# Following code inspired from item_KNN_based_CF.ipynb -> courtesy of RMIT.
K = 10  # Number of similar movies to consider
EPSILON = 1e-9
predictions = np.zeros((n_users, n_movies))
for i in range(n_movies):
    # Check if user i has rated the target movie
    if trainingDataset[i][random_movie] > 0:
        # Find top-K most similar movie to the target item based on Jaccard similarity
        similar_movie_ids = np.argsort(np_movie_jaccard_similarity[random_movie])[::-1][:K]

        # Coefficient values of similar movie
        similar_values = np_movie_jaccard_similarity[random_movie][similar_movie_ids]

        # Average rating of the target movie
        movie_rating_mean = np.sum(trainingDataset.T.values[random_movie]) / (
            np.sum(np.clip(trainingDataset.T.values[random_movie], 0, 1)) + EPSILON
        )

        # Ratings of similar movie for user i
        similar_movies = trainingDataset.T.values[similar_movie_ids]
        similar_movies_mean = np.sum(similar_movies, axis=1) / (
            np.sum(np.clip(similar_movies, 0, 1), axis=1) + EPSILON
        )

        # Calculate the predicted rating for user i and the target movie
        similar_sum_mean = similar_values * (similar_movies[:, i] - similar_movies_mean)
        weight = np.clip(similar_movies[:, i], 0, 1)
        similar_sum_mean *= weight

        predictions[i][random_movie] = movie_rating_mean + np.sum(similar_sum_mean) / (np.sum(similar_values * weight) + EPSILON)
        predictions[i][random_movie] = np.clip(predictions[i][random_movie], 0, 5)
        
## RMSE on Testing set (Jaccard  similarity)

rmse = np.sqrt(mean_squared_error(testingDataset.values[random_movie, :], predictions[random_movie, :]))
print("RMSE (Item-based): " + str(rmse));

RMSE (Item-based): 0.384316590493689


In [33]:
np_movie_cosine_similarity = np.zeros((n_movies, n_movies))

# Implementing Cosine Similarity. 

In [34]:
from sklearn.metrics.pairwise import cosine_similarity

for i, item_i in enumerate(trainingDataset.T.values):
    for j, item_j in enumerate(trainingDataset.T.values):
        
        # Calculate cosine similarity between item_i and item_j
        np_movie_cosine_similarity[i][j] =  cosine_similarity([item_i], [item_j])[0][0]

In [35]:
# Following code inspired from item_KNN_based_CF.ipynb -> courtesy of RMIT.
K = 10  # Number of similar movies to consider
EPSILON = 1e-9
predictions = np.zeros((n_users, n_movies))

for i in range(n_movies):
    # Check if user i has rated the target movie
    if trainingDataset[i][random_movie] > 0:
        # Find top-K most similar movie to the target item based on cosine similarity
        similar_movie_ids = np.argsort(np_movie_cosine_similarity[random_movie])[::-1][:K]

        # Coefficient values of similar movie
        similar_values = np_movie_cosine_similarity[random_movie][similar_movie_ids]

        # Average rating of the target movie
        movie_rating_mean = np.sum(trainingDataset.T.values[random_movie]) / (
            np.sum(np.clip(trainingDataset.T.values[random_movie], 0, 1)) + EPSILON
        )

        # Ratings of similar movie for user i
        similar_movies = trainingDataset.T.values[similar_movie_ids]
        similar_movies_mean = np.sum(similar_movies, axis=1) / (
            np.sum(np.clip(similar_movies, 0, 1), axis=1) + EPSILON
        )

        # Calculate the predicted rating for user i and the target movie
        similar_sum_mean = similar_values * (similar_movies[:, i] - similar_movies_mean)
        weight = np.clip(similar_movies[:, i], 0, 1)
        similar_sum_mean *= weight

        predictions[i][random_movie] = movie_rating_mean + np.sum(similar_sum_mean) / (np.sum(similar_values * weight) + EPSILON)
        predictions[i][random_movie] = np.clip(predictions[i][random_movie], 0, 5)
        
# RMSE on Testing set (Cosine  similarity)
rmse = np.sqrt(mean_squared_error(testingDataset.values, predictions))
print("RMSE (Item-based): " + str(rmse));

RMSE (Item-based): 0.3843205341548034


# Task 3: A Better Recommender System: OptionRecSys1

Task 3.1

In [36]:
# Following code inspired from item_KNN_based_CF.ipynb -> courtesy of RMIT.
DELTA = 25
EPSILON = 1e-9

np_user_pearson_corr = np.zeros((n_users, n_users))

for i, user_i in enumerate(trainingDataset.values):
    for j, user_j in enumerate(trainingDataset.values):
        
        # Creation of two boolean masks. 
        # Masks filters for all ratings from a user that's over zero. 
        # Ratings by the current pair of users
        mask_i = user_i > 0
        mask_j = user_j > 0

        # Common items rated by both users
        #Getting the intersection. All the movies where both users I and J watched and rated.
        corrated_index = np.intersect1d(np.where(mask_i), np.where(mask_j))
        if len(corrated_index) == 0:
            continue

        # Average value of user i and user j
        # getting the mean rating for both users.
        mean_user_i = np.sum(user_i) / (np.sum(np.clip(user_i, 0, 1)) + EPSILON)
        mean_user_j = np.sum(user_j) / (np.sum(np.clip(user_j, 0, 1)) + EPSILON)

        # Computing Pearson correlation
        
        # minusing the intersection by the mean. 
        user_i_sub_mean = user_i[corrated_index] - mean_user_i
        user_j_sub_mean = user_j[corrated_index] - mean_user_j

        #Squaring the sub means
        r_ui_sub_ri_sq = np.square(user_i_sub_mean)
        r_uj_sub_rj_sq = np.square(user_j_sub_mean)
        
        #getting the sum and square rooting the submeans.
        r_ui_sub_ri_sq_sum_sqrt = np.sqrt(np.sum(r_ui_sub_ri_sq))
        r_uj_sub_rj_sq_sum_sqrt = np.sqrt(np.sum(r_uj_sub_rj_sq))
        
        #calculating the similarity score. 
        sim = np.sum(user_i_sub_mean * user_j_sub_mean) / (r_ui_sub_ri_sq_sum_sqrt * r_uj_sub_rj_sq_sum_sqrt + EPSILON)

        # Significance weighting
        np_user_pearson_corr[i][j] = (min(len(corrated_index), DELTA) / DELTA) * sim

        

In [37]:
np_user_pearson_corr

array([[ 1.        , -0.06611845, -0.10189333, ...,  0.        ,
         0.02637021,  0.11399329],
       [-0.06611845,  1.        ,  0.06006731, ..., -0.08409993,
         0.11496231, -0.15785952],
       [-0.10189333,  0.06006731,  1.        , ...,  0.00948718,
         0.09284253, -0.14097359],
       ...,
       [ 0.        , -0.08409993,  0.00948718, ...,  0.56      ,
        -0.00581789, -0.14864572],
       [ 0.02637021,  0.11496231,  0.09284253, ..., -0.00581789,
         1.        ,  0.34925633],
       [ 0.11399329, -0.15785952, -0.14097359, ..., -0.14864572,
         0.34925633,  1.        ]])

In [38]:
# Following code inspired from item_KNN_based_CF.ipynb -> courtesy of RMIT.
K = 1
EPSILON = 1e-9

# Create an array to store predictions for user 1949
target_user_predictions = np.zeros_like(testingDataset.values[random_user])

for j in range(n_movies):
    if testingDataset.values[random_user, j] > 0:
        # Find top-K most similar users to the target user, excluding itself
        similar_user_ids = np.argsort(np_user_pearson_corr[random_user])[-(K + 1):-1]

        # Generates the coefficient values of similar users
        similar_values = np_user_pearson_corr[random_user][similar_user_ids]

        # Calculates the average value of the target user's ratings
        similar_users = trainingDataset.values[similar_user_ids]
        user_mean = np.sum(trainingDataset.values[random_user]) / (np.sum(np.clip(trainingDataset.values[random_user], 0, 1)) + EPSILON)
        similar_user_mean = np.sum(similar_users, axis=1) / (np.sum(np.clip(similar_users, 0, 1), axis=1) + EPSILON)

        # Select the users who rated item j
        users_who_rated_J = similar_users[:, j] > 0
        
        sum_mean = similar_values[users_who_rated_J] * (similar_users[users_who_rated_J, j] - similar_user_mean[users_who_rated_J])

        # Filter unrated items
        weight = np.clip(similar_users[users_who_rated_J, j], 0, 1)
        sum_mean *= weight

        # Calculate the predicted rating for item j by the target user
        target_user_predictions[j] = user_mean + np.sum(sum_mean) / (np.sum(similar_values[users_who_rated_J]) + EPSILON)
        target_user_predictions[j] = np.clip(target_user_predictions[j], 0, 5)

# Actual ratings for the target user
actual_ratings = testingDataset.values[random_user, :]

# Calculate RMSE for the target user's ratings
mse = mean_squared_error(actual_ratings, target_user_predictions)
rmse = np.sqrt(mse)

print(f"RMSE for user {random_user} when K = {K}: {rmse}")

RMSE for user 1949 when K = 1: 0.11354582624158482


In [39]:
# Following code inspired from item_KNN_based_CF.ipynb -> courtesy of RMIT.
K = 10
EPSILON = 1e-9

# Create an array to store predictions for user 1949
target_user_predictions = np.zeros_like(testingDataset.values[random_user])

for j in range(n_movies):
    if testingDataset.values[random_user, j] > 0:
        # Find top-K most similar users to the target user, excluding itself
        similar_user_ids = np.argsort(np_user_pearson_corr[random_user])[-(K + 1):-1]

        # Generates the coefficient values of similar users
        similar_values = np_user_pearson_corr[random_user][similar_user_ids]

        # Calculates the average value of the target user's ratings
        similar_users = trainingDataset.values[similar_user_ids]
        user_mean = np.sum(trainingDataset.values[random_user]) / (np.sum(np.clip(trainingDataset.values[random_user], 0, 1)) + EPSILON)
        similar_user_mean = np.sum(similar_users, axis=1) / (np.sum(np.clip(similar_users, 0, 1), axis=1) + EPSILON)

        # Select the users who rated item j
        users_who_rated_J = similar_users[:, j] > 0
        
        sum_mean = similar_values[users_who_rated_J] * (similar_users[users_who_rated_J, j] - similar_user_mean[users_who_rated_J])

        # Filter unrated items
        weight = np.clip(similar_users[users_who_rated_J, j], 0, 1)
        sum_mean *= weight

        # Calculate the predicted rating for item j by the target user
        target_user_predictions[j] = user_mean + np.sum(sum_mean) / (np.sum(similar_values[users_who_rated_J]) + EPSILON)
        target_user_predictions[j] = np.clip(target_user_predictions[j], 0, 5)

# Actual ratings for the target user
actual_ratings = testingDataset.values[random_user, :]

mse = mean_squared_error(actual_ratings, target_user_predictions)
rmse = np.sqrt(mse)

print(f"RMSE for user {random_user} when K = {K}: {rmse}")

RMSE for user 1949 when K = 10: 0.08695366109121724


In [40]:
# Following code inspired from item_KNN_based_CF.ipynb -> courtesy of RMIT.
K = 20
EPSILON = 1e-9

# Create an array to store predictions for user 1949
target_user_predictions = np.zeros_like(testingDataset.values[random_user])

for j in range(n_movies):
    if testingDataset.values[random_user, j] > 0:
        # Find top-K most similar users to the target user, excluding itself
        similar_user_ids = np.argsort(np_user_pearson_corr[random_user])[-(K + 1):-1]

        # Generates the coefficient values of similar users
        similar_values = np_user_pearson_corr[random_user][similar_user_ids]

        # Calculates the average value of the target user's ratings
        similar_users = trainingDataset.values[similar_user_ids]
        user_mean = np.sum(trainingDataset.values[random_user]) / (np.sum(np.clip(trainingDataset.values[random_user], 0, 1)) + EPSILON)
        similar_user_mean = np.sum(similar_users, axis=1) / (np.sum(np.clip(similar_users, 0, 1), axis=1) + EPSILON)

        # Select the users who rated item j
        users_who_rated_J = similar_users[:, j] > 0
        
        sum_mean = similar_values[users_who_rated_J] * (similar_users[users_who_rated_J, j] - similar_user_mean[users_who_rated_J])

        # Filter unrated items
        weight = np.clip(similar_users[users_who_rated_J, j], 0, 1)
        sum_mean *= weight

        # Calculate the predicted rating for item j by the target user
        target_user_predictions[j] = user_mean + np.sum(sum_mean) / (np.sum(similar_values[users_who_rated_J]) + EPSILON)
        target_user_predictions[j] = np.clip(target_user_predictions[j], 0, 5)

# Actual ratings for the target user
actual_ratings = testingDataset.values[random_user, :]

mse = mean_squared_error(actual_ratings, target_user_predictions)
rmse = np.sqrt(mse)

print(f"RMSE for user {random_user} when K = {K}: {rmse}")

RMSE for user 1949 when K = 20: 0.08120000718904473


In [41]:
# Following code inspired from item_KNN_based_CF.ipynb -> courtesy of RMIT.
K = 30
EPSILON = 1e-9

# Create an array to store predictions for user 1949
target_user_predictions = np.zeros_like(testingDataset.values[random_user])

for j in range(n_movies):
    if testingDataset.values[random_user, j] > 0:
        # Find top-K most similar users to the target user, excluding itself
        similar_user_ids = np.argsort(np_user_pearson_corr[random_user])[-(K + 1):-1]

        # Generates the coefficient values of similar users
        similar_values = np_user_pearson_corr[random_user][similar_user_ids]

        # Calculates the average value of the target user's ratings
        similar_users = trainingDataset.values[similar_user_ids]
        user_mean = np.sum(trainingDataset.values[random_user]) / (np.sum(np.clip(trainingDataset.values[random_user], 0, 1)) + EPSILON)
        similar_user_mean = np.sum(similar_users, axis=1) / (np.sum(np.clip(similar_users, 0, 1), axis=1) + EPSILON)

        # Select the users who rated item j
        users_who_rated_J = similar_users[:, j] > 0
        
        sum_mean = similar_values[users_who_rated_J] * (similar_users[users_who_rated_J, j] - similar_user_mean[users_who_rated_J])

        # Filter unrated items
        weight = np.clip(similar_users[users_who_rated_J, j], 0, 1)
        sum_mean *= weight

        # Calculate the predicted rating for item j by the target user
        target_user_predictions[j] = user_mean + np.sum(sum_mean) / (np.sum(similar_values[users_who_rated_J]) + EPSILON)
        target_user_predictions[j] = np.clip(target_user_predictions[j], 0, 5)

# Actual ratings for the target user
actual_ratings = testingDataset.values[random_user, :]

mse = mean_squared_error(actual_ratings, target_user_predictions)
rmse = np.sqrt(mse)

print(f"RMSE for user {random_user} when K = {K}: {rmse}")

RMSE for user 1949 when K = 30: 0.0911502660340976


In [42]:
# Following code inspired from item_KNN_based_CF.ipynb -> courtesy of RMIT.
K = 50
EPSILON = 1e-9

# Create an array to store predictions for user 1949
target_user_predictions = np.zeros_like(testingDataset.values[random_user])

for j in range(n_movies):
    if testingDataset.values[random_user, j] > 0:
        # Find top-K most similar users to the target user, excluding itself
        similar_user_ids = np.argsort(np_user_pearson_corr[random_user])[-(K + 1):-1]

        # Generates the coefficient values of similar users
        similar_values = np_user_pearson_corr[random_user][similar_user_ids]

        # Calculates the average value of the target user's ratings
        similar_users = trainingDataset.values[similar_user_ids]
        user_mean = np.sum(trainingDataset.values[random_user]) / (np.sum(np.clip(trainingDataset.values[random_user], 0, 1)) + EPSILON)
        similar_user_mean = np.sum(similar_users, axis=1) / (np.sum(np.clip(similar_users, 0, 1), axis=1) + EPSILON)

        # Select the users who rated item j
        users_who_rated_J = similar_users[:, j] > 0
        
        sum_mean = similar_values[users_who_rated_J] * (similar_users[users_who_rated_J, j] - similar_user_mean[users_who_rated_J])

        # Filter unrated items
        weight = np.clip(similar_users[users_who_rated_J, j], 0, 1)
        sum_mean *= weight

        # Calculate the predicted rating for item j by the target user
        target_user_predictions[j] = user_mean + np.sum(sum_mean) / (np.sum(similar_values[users_who_rated_J]) + EPSILON)
        target_user_predictions[j] = np.clip(target_user_predictions[j], 0, 5)

# Actual ratings for the target user
actual_ratings = testingDataset.values[random_user, :]

mse = mean_squared_error(actual_ratings, target_user_predictions)
rmse = np.sqrt(mse)

print(f"RMSE for user {random_user} when K = {K}: {rmse}")

RMSE for user 1949 when K = 50: 0.09448551994740487
